In [2]:
import os
import sys

#Pega o caminho absoluto para a pasta raiz do projeto (OBESITY_PROJECT)
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))

#Adiciona a pasta raiz (que contém 'src') ao caminho de busca do Python
if project_root not in sys.path:
    sys.path.append(project_root)
    
from src.utils.db_connection import get_data
import pandas as pd

In [3]:
#Query simples para ler a tabela silver do banco de dados
query = "SELECT * FROM obesity_silver"
df_silver = get_data(query)


#Visualizando as 5 primeiras linhas do dataframe
df_silver.head()

,genero,idade,altura_m,peso_kg,historico_familiar,consumo_frequente_alimentos_caloricos,consumo_frequente_vegetais,numero_refeicoes_principais_dia,consumo_lanches_entre_refeicoes,habito_fumar,consumo_diario_agua,monitora_caloria_diaria,frequencia_semanal_atividade_fisica,tempo_uso_dispositivo,consumo_bebida_alcoolica,transporte_habitual,classe_peso_corporal
0,Female,21,1.62,64.0,yes,no,2,3,Sometimes,no,2,no,0,1,no,Public_Transportation,Peso Normal
1,Female,21,1.52,56.0,yes,no,3,3,Sometimes,yes,3,yes,3,0,Sometimes,Public_Transportation,Peso Normal
2,Male,23,1.80,77.0,yes,no,2,3,Sometimes,no,2,no,2,1,Frequently,Public_Transportation,Peso Normal
3,Male,27,1.80,87.0,no,no,3,3,Sometimes,no,2,no,2,0,Frequently,Walking,Sobrepeso Nível I
4,Male,22,1.78,89.8,no,no,2,1,Sometimes,no,2,no,0,0,Sometimes,Public_Transportation,Sobrepeso Nível II
